# Moss Tool Example

This notebook demonstrates how to use the `MossToolSpec` with LlamaIndex.

## Installation

First, you need to install the package (and LlamaIndex core if not present).

In [ ]:
%pip install llama-index-tools-moss llama-index-core

## Setup

Import the necessary classes and initialize the Moss client.

In [ ]:
from llama_index.tools.moss import MossToolSpec
from inferedge_moss import MossClient, DocumentInfo

# Initialize Moss Client
client = MossClient()

# Initialize Tool Spec
moss_tool = MossToolSpec(
    client=client, 
    index_name="demo_index",
    top_k=3,
    alpha=0.5
)

## Indexing Data

Let's add some sample data to our index.

In [ ]:
docs = [
    DocumentInfo(
        text="LlamaIndex connects your data to LLMs.", 
        metadata={"topic": "AI"}
    ),
    DocumentInfo(
        text="Moss provides fast semantic search integration.", 
        metadata={"topic": "Search"}
    ),
]

# Index the documents
await moss_tool.index_docs(docs)

## Using with an Agent

Now we can wrap the tool for use with a LlamaIndex agent.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import MockLLM

# Convert to tool list
tools = moss_tool.to_tool_list()

# Create an agent (Using MockLLM for demonstration)
llm = MockLLM()
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

# Chat with the agent
response = await agent.achat("What does Moss provide?")
print(response)